In [ ]:
import os

import numpy as np
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from PIL import Image

from unet import get_unet, jaccard_coef_int, jaccard_coef_loss

# LOAD THE MODEL

In [ ]:
# set up the gpu id# set u 
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

In [ ]:
model = get_unet(3, 512, 512)

In [ ]:
adam = Adam(lr=1e-3)
model.compile(adam, loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int])

In [ ]:
# model.save('/root/data/models/unet/unet_app_test.h5')

# CREATE DATASET

In [ ]:
import json
import random
from pycocotools.mask import decode
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import cv2
from skimage.transform import resize
import glob
import os

In [ ]:
all_images = glob.glob('/root/data/fish_identification/basler0/*.jpg')

In [ ]:
labels = json.load(open('/root/data/fish_identification/new_annotations.json'))

In [ ]:
instruction = "Draw a bounding box around all fish or any part of a fish. It does not matter how far it is or if you only see a small part of it."

In [ ]:
labs = [l for l in labels if l["instructions"] == instruction]
print(len(labs))

In [ ]:
# no_poly = 0
# for l in labs:
#     # get the image path
#     trunc_path = l['content'][-10:]
#     try:
#         img_path = glob.glob('/root/data/fish_identification/basler0/*'+trunc_path)[0]
#         width, height = Image.open(img_path).size
#         # create empty Image
#         masks = Image.new('L', (width, height), 0)

#         # load the polygons 
#         polygons = l['results']
#         if len(polygons) > 0:
#             for polygon in polygons:
#                 good_polygon = [tuple([poly[0], 3000-poly[1]]) for poly in polygon]
#                 ImageDraw.Draw(masks).polygon(good_polygon, outline='red', fill='red')
#             f, ax = plt.subplots(1, 2, figsize=(10, 5))
#             ax[0].imshow(np.array(Image.open(img_path)))
#             ax[1].imshow(np.array(masks))
#             plt.show()
#             print(img_path)
#         else: 
#             no_poly += 1
#     except Exception as e:
#         print e.message

In [ ]:
with open('/root/data/fish_identification/good_fish.txt', 'r') as f:
    good = []
    for line in f:
        good.append(line[:-1])

In [ ]:
no_poly = 0
for l in labs:
    # get the image path
    trunc_path = l['content'][-10:]
    img_paths = glob.glob('/root/data/fish_identification/basler0/*'+trunc_path)
    if len(img_paths) > 0 :
        img_path = img_paths[0]
        width, height = Image.open(img_path).size
        if img_path in good:
            masks = Image.new('L', (width, height), 0)
            polygons = l['results']
            for polygon in polygons:
                good_polygon = [tuple([poly[0], 3000-poly[1]]) for poly in polygon]
                ImageDraw.Draw(masks).polygon(good_polygon, outline='white', fill='white')
            masks_name = os.path.basename(img_path).split('.')[0].split('_')[-1]
            masks = np.array(masks)
            masks[masks>0]=1
            np.save('/root/data/fish_identification/masks/' + masks_name, masks)
            
#         # create empty Image
#         masks = Image.new('L', (width, height), 0)

#         # load the polygons 
#         
#         if len(polygons) > 0:
#             for polygon in polygons:
#                 good_polygon = [tuple([poly[0], 3000-poly[1]]) for poly in polygon]
#                 ImageDraw.Draw(masks).polygon(good_polygon, outline='red', fill='red')
#             f, ax = plt.subplots(1, 2, figsize=(10, 5))
#             ax[0].imshow(np.array(Image.open(img_path)))
#             ax[1].imshow(np.array(masks))
#             plt.show()
#             print(img_path)
#         else: 
#             no_poly += 1
#     except Exception as e:
#         print e.message

In [ ]:
import cv2

In [ ]:
plt.imshow(cv2.resize(masks, (512, 512)))

# CREATE GENERATOR


In [ ]:
import json
import random
from pycocotools.mask import decode
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import cv2
from skimage.transform import resize
import glob
import random
import os

In [ ]:
with open('/root/data/fish_identification/good_fish.txt', 'r') as f:
    good = []
    for line in f:
        good.append(line[:-1])

In [ ]:
good = [g for g in good if len(g)>0]
random.shuffle(good)
train_set = good[:70]
val_set = good[70:]

In [ ]:
val_set

In [ ]:
batch_size = 8
steps_per_epoch = len(good) // batch_size
input_shape = (512, 512, 3)

In [ ]:
# def build_truth(masks, input_shape):
#     masks = decode(masks)
#     mask = np.sum(masks, axis=2)
#     # return mask[x:x+input_shape[0], y:y+input_shape[1]]
#     mask = resize(mask, (input_shape[0], input_shape[1]))
#     mask[mask>0] = 1
#     return mask

In [ ]:
def generator(labels, steps_per_epoch, BATCH_SIZE, input_shape):
    i = 0
    img_size = input_shape[0]
    while True:
        x_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], input_shape[2]), dtype=np.uint8)
        y_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], 1), dtype=np.uint8)
        for (ind, j) in enumerate(range(i*BATCH_SIZE, (i+1)*BATCH_SIZE)):
            img_path = np.random.choice(labels)
            masks_name = os.path.basename(img_path).split('.')[0].split('_')[-1]
            mask_path = '/root/data/fish_identification/masks/{}.npy'.format(masks_name)
            x_batch[ind,...] = np.array(Image.open(img_path).resize((input_shape[0], input_shape[1])))
            y_batch[ind,...] = np.expand_dims(cv2.resize(np.load(mask_path), 
                                                         (input_shape[0], input_shape[1])), 
                                              axis =2)
        i += 1
        if i >= steps_per_epoch:
            i = 0
        yield x_batch, y_batch

In [ ]:
# x = 100
# y = 100
# np.array(Image.open(labels[0]['path']))[x:x+input_shape[0], y:y+input_shape[1]]

In [ ]:
train_generator = generator(train_set, steps_per_epoch, batch_size, input_shape)
validation_generator = generator(val_set, steps_per_epoch, batch_size, input_shape)

In [ ]:
xb, yb = train_generator.next()

In [ ]:
xb.dtype

In [ ]:
plt.imshow(xb[0,...])

In [ ]:
xb.dtype

In [ ]:
plt.imshow(yb[0,...,0])
plt.colorbar()

In [ ]:
imgg= Image.fromarray(xb[0,...])
maskk = Image.fromarray(yb[0,...,0])

In [ ]:
imgg.size

In [ ]:
maskk.size

In [ ]:
plt.imshow(xb[0,...])
plt.imshow(yb[0,...,0], alpha=0.2)

# TRAIN

In [ ]:
from keras.callbacks import Callback

In [ ]:
# create history callback
class SaveHistory(Callback):
    
    def __init__(self, json_path):
        self.json_path = json_path
    
    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        with open(self.json_path, 'w') as f:
            json.dump(self.history, f)

In [ ]:
saveh = SaveHistory('./imenco_history.json')

In [ ]:
# start training# start 
history = model.fit_generator(
        generator=train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=50,
        verbose=1,
        validation_data=validation_generator,
        validation_steps=steps_per_epoch//10,
        callbacks=[saveh])

# plot history

In [ ]:
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
plt.plot(history.history['jaccard_coef_int'], label='iou')
plt.plot(history.history['val_jaccard_coef_int'], label='val_iou')
plt.legend()

# PREDICTION

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
gt = np.load('/root/data/fish_identification/masks/0623.npy')
gt = cv2.resize(gt, (512, 512))
plt.imshow(gt)

In [ ]:
imp = np.array(Image.open('/root/data/fish_identification/basler0/Basler acA4112-8gc (40003213)_20180326_134801204_0623.jpg').resize((512, 512)))

In [ ]:
plt.imshow(imp)

In [ ]:
out = model.predict(np.expand_dims(imp, axis=0))

In [ ]:
out.shape

In [ ]:
plt.imshow(out[0,...,0])
plt.colorbar()
# plt.clim([0, 0.0001])

In [ ]:
np.count_nonzero(out==1)

In [ ]:
import copy

In [ ]:
img = Image.fromarray(imp)
mask = np.zeros((512, 512, 3), dtype=np.uint8)
tmp = copy.deepcopy(out[0,...,0])
tmp[tmp>0.1] = 255
mask[:,:,0] = tmp
mask = Image.fromarray(mask)

In [ ]:
Image.blend(img, mask, 0.5)